# Theme analysis in Rap Songs

After going through my course content on Natural language processing I decided to give a try.
We willl train on Rap Songs.

## Goals
* Build a list of rappers (might be by coast or other)

* Build a webscrapper to extract lyrics form Genius using their api

* Natural langage processing with collected data

    1- use Tfidfvectorizer to fit transform and create a sparse matrix (tfidf)
    
    2- construct NMF based on sparse matrix to analyze thematics
    
    3- (optional) use kmeans to identify clusters 
        a- build pipeline with svd and kmeans
        b- fit and predict on sparse matrix labels

In [1]:
import pandas as pd
import urllib3
from urllib.parse import quote
import json

from bs4 import BeautifulSoup
import re
from io import StringIO

# Import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Import NMF
from sklearn.decomposition import NMF

from sklearn.feature_extraction import text

import warnings
warnings.filterwarnings("ignore")

## Build a list of rappers (might be by coast or other)

In [2]:
#all recognized hip hop artists from wikipedia

init_lib2 = urllib3.PoolManager()
#wp2 = init_lib2.request('GET', 'https://en.wikipedia.org/wiki/List_of_hip_hop_musicians', assert_same_host=False)
wp2 = init_lib2.request('GET', 'https://en.wikipedia.org/wiki/List_of_West_Coast_hip_hop_artists')
html2 = BeautifulSoup(wp2.data, "html.parser") # Extract the page's HTML as a string

b = ""
for i in range(0,27):
    a = html2.find_all("div", class_="div-col columns column-width")[i].get_text()
    b = b + a

rappers = pd.read_csv(StringIO(b), sep='\t', header=None)

#flatten rappers list
k = rappers.values.tolist()
flatrap = []
for n1 in k:
    for n2 in n1:
        flatrap.append(n2)
flatrap[0]

'100s'

## Build a webscrapper to extract lyrics form Genius using their api

In [3]:
#client key
cl_TK2 = 'X7D7eCcQjbLn9_clVs0mzhB-vg3bQv3-30zmDVVRbJW07w6m0YQHhAdSUvvJo9BV'
base_url = "https://api.genius.com/"
gen_search = "search?q="
art_search = "artists/"
songs_search = "songs/"

In [4]:
# define url scrapper from genius api
def search_title(search):
    #build query
    req = base_url + gen_search + quote(search) # build str query
    http = urllib3.PoolManager() # instantiate a poolmanager
    request = http.request('GET', req, headers={"Authorization":"Bearer " + cl_TK2, "User-Agent": ""}) # request with heqder

    data = json.loads(request.data.decode('utf-8'))
        
    title = []
    for i in range(0, len(data['response']['hits'])):
        title.append(data['response']['hits'][i]['result']['title'])
    
    return title

In [5]:
# define url scrapper from genius api
def search_urls(search):
    #build query
    req = base_url + gen_search + quote(search) # build str query
    http = urllib3.PoolManager() # instantiate a poolmanager
    request = http.request('GET', req, headers={"Authorization":"Bearer " + cl_TK2, "User-Agent": ""}) # request with heqder

    data = json.loads(request.data.decode('utf-8'))
        
    url = []
    for i in range(0, len(data['response']['hits'])):
        url.append(data['response']['hits'][i]['result']['url'])
    
    return url

In [6]:
# lyrics scrapper based on url
def search_lyrics(url):    
    # collect lyrics    
    init_lib = urllib3.PoolManager()
    wp = init_lib.request('GET', url)
    html = BeautifulSoup(wp.data, "html.parser") # Extract the page's HTML as a string

    # Scrape lyrics from webpage
    lyrics = html.find("div", class_="lyrics").get_text()
    return lyrics

In [7]:
def geniusscrapper(search):
    r = search_urls(search)
    
    scrap = []
    for i, j in enumerate(r):
        l = search_lyrics(j)
        scrap.append(l)
    return scrap

In [8]:
# trial with Kendrick Lamar
#kdot = geniusscrapper('Eminem')
#print(ktit)

In [9]:
db = []
songtitle = []
art = []
for r, n in enumerate(flatrap):
    
    a = geniusscrapper(n)
    b = search_title(n)
    
    art.extend(n)
    db.extend(a)
    songtitle.extend(b)

In [10]:
print(len(db))
print(len(songtitle))
print(len(art))

adb = pd.DataFrame(db)
asongtitle = pd.DataFrame(songtitle)
aart = pd.DataFrame(art)

adb.to_csv('adb.csv', index=True)
asongtitle.to_csv('asongtitle.csv', index=True)
aart.to_csv('aart.csv', index=True)

6085
6085
6336
